In [1]:
!pip install evaluate
!pip install rouge_score
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 595.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=72217254bf86f1c4f42a17f13730cffb19a444da1afc8f8e17e308d5c1b43e19
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfu

In [2]:
import os, sys, random, re, collections, string

try:
    from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset

import numpy as np
import re

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import BCELoss

from tqdm.auto import tqdm

import sklearn.model_selection
import sklearn.metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import heapq

import matplotlib.pyplot as plt

import transformers
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

try:
    import evaluate
except:
    !pip install evaluate
    !pip install rouge_score
    import evaluate


In [3]:
# https://huggingface.co/datasets/cnn_dailymail
dataset = load_dataset('cnn_dailymail', '3.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
modelname = 'distilbert/distilgpt2'
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained('distilgpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('distilbert/distilgpt2')


# tokenizer = AutoTokenizer.from_pretrained(modelname)
# model = AutoModelForSeq2SeqLM.from_pretrained(modelname)
model = gpt2_model
tokenizer = gpt2_tokenizer
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=modelname)
rouge = evaluate.load("rouge")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
#you can then get the individual splits
train = dataset['train'].select(range(10000))
test = dataset['test'].select(range(1000))
validation = dataset['validation']

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

train

# gpt2_tokenizer.to(device)
# gpt2_model.to(device)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 10000
})

In [6]:
# ctr = len(train)
# i = 0
# j = 0
# for dat in train:
#     i+=1
#     # print(i, len(dat['article'].split(' -- ')[1]))
#     # print(i, dat['article'].count('--'))
#     splitstr = re.split(r' -- ', dat['article'], 1)
#     if (re.search(r'(\([\w+]+\))|([A-Z][A-Z]+)|(\([\w\s]+\))|(\([\w]+\.[\w]+\))', splitstr[0])) is not None:
#         # print(i, splitstr)
#         j+=1
#     if i > ctr:
#         break

# print(j, j/ctr)

In [7]:
class cnnDataSet(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.id = self.data['id']
        self.article = self.data['article']
        self.highlight = self.data['highlight']

    def get_article(text):
        splittext = text.split(" -- ")[1]




In [8]:
def trainModel(model):
    pass

In [9]:
def split_article(art):
    splitstr = re.split(' -- ', art, 1)
    # print(splitstr)
    if len(splitstr) == 1 or (re.search(r'(\([\w+]+\))|([A-Z][A-Z]+)|(\([\w\s]+\))|(\([\w]+\.[\w]+\))', splitstr[0])) is None:
        return art
    else:
        return splitstr[1]


def tokenize_str(examples):
    # inputs = [doc for doc in examples["article"]]
    inputs = []
    labs = examples['highlights']
    # print(len(labs))
    # print(len(examples["article"]))
    ctr = 0
    # for i, x in enumerate(examples["article"]):
    #     art = split_article(x)

    #     inputs.append(art)

    inputs = [split_article(art) for art in examples['article']]

    # model_inputs = gpt2_tokenizer(inputs, max_length=1024, truncation=True)
    # labels = gpt2_tokenizer(text_target=labs, max_length=128, truncation=True)

    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=labs, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [10]:
tokenized_train_dataset = train.map(tokenize_str, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
tokenized_test_dataset = test.map(tokenize_str, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = gpt2_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [16]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir="cnn_trained_"+modelname,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ValueError: Expected input batch_size (5115) to match target batch_size (335).